# Resolution of the optimisation problem

This notebook will help solve the following optimisation problem : 
TODO

In [19]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)


PROJECT_ROOT = c:\Users\HERVET\OneDrive - CentraleSupelec\Documents\FMGP\avencore_hydrogen_production


## 1. Imports initiaux

In [ ]:
import pyomo.environ as pyo

from Data.loading import load_prices
from Battery.battery_simulation import BatteryParams, BatteryBlock
from Electrolyser.electrolyser_simulation import ElectrolyserParams, ElectrolyserBlock, PowerBalanceBlock
from Costs.cost_functions import EconomicParams, ObjectiveBuilder

## 2. Loading the dataset

In [21]:
PATH_SPOT = "../Data/prix_spot.csv"
ELEC_PRICE, CO2_INTENSITY, T, df_2024 = load_prices(PATH_SPOT, year=2024)

print(df_2024.head())

                      Date  Spot  T_weight  W_100
192840 2024-01-01 00:00:00  0.10       6.7   33.0
192841 2024-01-01 01:00:00  0.01       6.7   31.0
192842 2024-01-01 02:00:00  0.00       6.6   30.0
192843 2024-01-01 03:00:00 -0.01       6.4   30.0
192844 2024-01-01 04:00:00 -0.03       6.3   29.0


## 3. Global parameters

In [22]:
dt = 1.0
ECON_PARA = EconomicParams(
    CAPEX_PWR_BAT = 150.0,
    CAPEX_EN_BAT = 10_000.0,
    FORWARD_PRICE = 75.0,
    CO2_COST = 80.0
)

ELECTRO_PARA = ElectrolyserParams(
    PWR_H2_MAX = 100.0,
    U_ELECTRO_MIN = 0.10,
    U_ELECTRO_MAX = 0.95,
    RAMP_ELECTRO = 1.0,
    ELECTRO_YIELD = 0.70,
    LHV_H2 = 55.0,
    dt = dt,
    H2_TARGET = 1_000_000.0
)

BAT_PARA = BatteryParams(
    CHARGE_YIELD = 1.0,
    DISCHARGE_YIELD = 1.0,
    dt = dt,
    SOC_MIN = 0.10,
    SOC_MAX = 0.95
)

## 4. Pyomo model definition

In [23]:
m = pyo.ConcreteModel()
m.T = pyo.RangeSet(0, T-1)

# Dimension variables
m.PWR_BAT_MAX = pyo.Var(domain=pyo.NonNegativeReals) # MW
m.EN_BAT_MAX = pyo.Var(domain=pyo.NonNegativeReals) # MWh

## 5. Adding the different blocs

In [24]:
# Battery
bat = BatteryBlock(m, m.T, BAT_PARA)
bat.add_variables()
bat.add_constraints()

# PEM Electrolyser
el = ElectrolyserBlock(m, m.T, ELECTRO_PARA)
el.add_variables()
el.add_constraints()

# Power balance
pb = PowerBalanceBlock(m, m.T, PWR_H2_MAX=ELECTRO_PARA.PWR_H2_MAX, PHI_BOUNDS=(0.0,1.0))
pb.add_variables()
pb.add_constraints()

## 6. Objective definition and solving

In [2]:
objb = ObjectiveBuilder(m, m.T, ECON_PARA, ELEC_PRICE=ELEC_PRICE, CO2_INTENSITY=CO2_INTENSITY, dt=dt, include_co2=False)
objb.build_objective()

solver = pyo.SolverFactory("highs")
res = solver.solve(m, tee=True)
print(res.solver.status, res.solver.termination_condition)

NameError: name 'ObjectiveBuilder' is not defined

## 7. Results

In [ ]:
MEAN_H2_PWR = sum(pyo.value(m.PWR_H2[t]) for t in m.T) / T
TOT_H2_PWR = sum(pyo.value(m.PWR_H2[t]) for t in m.T)
H2_TOT_PROD  = sum(pyo.value(m.H2[t]) for t in m.T)
COST_OPTI = pyo.value(m.Obj)
MEAN_COST = (sum(ECON_PARA.FORWARD_PRICE * pyo.value(m.PHI) * dt + ELEC_PRICE[t] * pyo.value(m.PWR_SPOT[t]) * dt for t in m.T) / max(TOT_H2_PWR, 1e-9))
LCOH_OPTI = COST_OPTI / max(H2_TOT_PROD, 1e-9)

print("Electrolyser max power (MW):", f"{ELECTRO_PARA.PWR_H2_MAX:.2f}")
print("Optimal battery power (MW):", f"{pyo.value(m.PWR_BAT_MAX):.2f}")
print("Optimal battery capacity (MWh):", f"{pyo.value(m.EN_BAT_MAX):.2f}")
print("Optimal percentage forward (%):", f"{pyo.value(m.PHI)*100:.2f}")
print("Mean power electrolyser (MW):", f"{MEAN_H2_PWR:.2f}")
print("Total H2 production (kg):", f"{H2_TOT_PROD:.2f}")
print("Total cost (€):", f"{COST_OPTI:.2f}")
print("LCOH (€/kg):", f"{LCOH_OPTI:.2f}")

ERROR: evaluating object as numeric value: PWR_H2[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized VarData object PWR_H2[0]


ValueError: No value for uninitialized VarData object PWR_H2[0]